In [44]:
import re
file="/home/davide/Desktop/genomiChro/intergeniche_RefSeq/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen.fasta"
import re

# Define the motifs
motivi= [
    ".{4}AG[ATCG].{4,11}ACT.{4}",
    ".{4}AGT.{4,11}A[AT]T.{4}",
    ".{4}AGT.{4,11}AC[AT].{4}",
    ".{4}AGT.{4,11}[TG]CT.{4}",
    ".{4}TGT.{4,11}AC[AT].{4}"
]
from Bio import SeqIO
#parse the fasta file
diz={}
j=0
for record in SeqIO.parse(file, "fasta"):
    seq=str(record.seq)
    #use the finditer method to find all the matches of the regular expression
    for m in motivi:
        for match in re.finditer(m, str(seq)):
            x=match.group()
            s,e=match.start(),match.end()
            #x=x[:4].lower()+x[4:-4]+x[-4:].lower()
            # print(x,s,e,seq[s],seq[e-1])
            #store the match in a dictionary
            if record.id in diz:
                diz[record.id].append(x)
            else:
                diz[record.id]=[x]
import pandas as pd
df=pd.Series(diz,name="motivi")

In [45]:
df

LAU37_RS00015    [AAACAGTACAAAACCCAATTTGGC, CATCAGTGTCTGCATCCAA...
LAU37_RS00030    [ATGTAGAATTCGGACTTATA, ATCAAGAAAATACCTGAACTTAG...
LAU37_RS00035                           [TAACAGCTTAATGAGGGACTTTAA]
LAU37_RS00055    [TATGAGTTCAATACTAAGG, TAATAGTGAGTATTTACCAATCCA...
LAU37_RS00060    [CTGTAGAGCAGCCCATACTTTTT, TGTTAGTTTTGCTTTACTGA...
                                       ...                        
LAU37_RS31375                            [TCGCTGTAGCAGAAGTACAAAAA]
LAU37_RS31380    [TTGAAGACAGCGCTAAACTTTGA, ATCAAGTTGTTGAATAGCA,...
LAU37_RS31390    [CCATAGGCCAGCCCACTCGTG, TCGTAGACTACTGCCCTCACTG...
LAU37_RS31405                             [GTTGAGTTTTTCTGGAATGCAT]
LAU37_RS31410    [TGCGAGAAACTGTCAGAACTGGCT, GTTGAGGGGAGAACCCAAC...
Name: motivi, Length: 4610, dtype: object

In [46]:
def revcomp(seq):
    seq=seq[::-1]
    diz={'a':'t','t':'a','c':'g','g':'c','A':'T','T':'A','C':'G','G':'C'}
    seq="".join([diz[x] for x in seq])
    return seq


In [47]:
s="tatcTGTTTCGCACAaata"
rev=revcomp(s)
#find best alignment between s and revcomp(s) where the alignment is done by shifting the two sequences and computing the number of matches

def best_alignment(s1, s2):
    best = 0
    best_alignment = ""
    shift=0
    for i in range(len(s1)):
        matches = 0
        temp_alignment = ""
        for j in range(len(s2)):
            if i + j < len(s1):
                if s1[i + j] == s2[j]:
                    matches += 1
                    temp_alignment += s1[i + j]
                else:
                    temp_alignment += '-'
        if matches > best:
            best = matches
            best_alignment = temp_alignment
            shift=i
    return best, best_alignment,shift


In [48]:
best_alignment(s,rev)
#now make a score for the alignment and print the clusters of aignment considering at most one mismatch and at least 3 matches for cluster, at least 2 clusters

(12, 'tat-TGT-----ACA-ata', 0)

In [49]:
#order df for the number of motifs df is a pandas series
#transform df in a pandas dataframe
df=pd.DataFrame(df)
#add a column with the number of motifs
df["n_motivi"]=df["motivi"].apply(len)
#order the dataframe
df=df.sort_values("n_motivi",ascending=False)
#drop the column with the number of motifs
df=df.drop("n_motivi",axis=1)
df

,motivi
LAU37_RS24005,"[ATTGAGGCTTGATTAACTACTTCAG, TATAAGAAATTTCTGAGA..."
LAU37_RS24030,"[TTTTAGTATTATTTGCTGACTACTT, AGTTAGCCAATCGCTGCT..."
LAU37_RS00155,"[TGACAGTACAGATACACAACTCTTC, GGCTAGGTGTCTACAACT..."
LAU37_RS11065,"[ATGCAGCGCAGTAAAACTCTAT, CTTCAGTTGTTATTTAACTTC..."
LAU37_RS12260,"[CGTTAGTTTTTATTACTAGAT, CTAAAGGTTCAAAAACTCGGC,..."
...,...
LAU37_RS04180,[CACTTGTTTTGGAGACTGAAT]
LAU37_RS27305,[GCCAAGACACCTAAACTTGGC]
LAU37_RS09050,[TGACAGGAAATAGCACTAATA]
LAU37_RS27350,[CGATAGTTGAAGGCTAAGA]


In [50]:
seqs=df.loc["LAU37_RS14470"]
for s in seqs.iloc[0]:
    rev=revcomp(s)
    score,alignment,shift=best_alignment(s,rev)
    print(s,rev,score,alignment,shift,sep="\n")
    print()

TGCCAGTGGGGGCATTTTTG
CAAAAATGCCCCCACTGGCA
6
----A-TG----CA-T----
0

TTGTAGTACGAATGTTCTATAG
CTATAGAACATTCGTACTACAA
12
-T-TAG-AC----GT-CTA-A-
0

TTTGTGTGTCACCTGACGACAAATT
AATTTGTCGTCAGGTGACACACAAA
12
--T-TGTC--C-G--GACA-A--
2


In [66]:
def clusters(alignment):
    clusters=[]
    cluster=[]
    for i,x in enumerate(alignment):
        # print(cluster)
        if x!="-":
            cluster.append(x)
        elif (i != len(alignment) - 1 and alignment[i+1]=="-") or i==len(alignment)-1:
            clusters.append(cluster)
            print(cluster,clusters)
            cluster=[]
        else:
            pass
    clusters=[x for x in clusters if len(x)>=3]
    return clusters

alignment="-T-TAG-AC----GT-CTA-A-"
clusters(alignment)
        

['T', 'T', 'A', 'G', 'A', 'C'] [['T', 'T', 'A', 'G', 'A', 'C']]
[] [['T', 'T', 'A', 'G', 'A', 'C'], []]
[] [['T', 'T', 'A', 'G', 'A', 'C'], [], []]
['G', 'T', 'C', 'T', 'A', 'A'] [['T', 'T', 'A', 'G', 'A', 'C'], [], [], ['G', 'T', 'C', 'T', 'A', 'A']]


[['T', 'T', 'A', 'G', 'A', 'C'], ['G', 'T', 'C', 'T', 'A', 'A']]